# Building a Spam filter with Naive Bayes

## Introduction
- In this project, we're going to build a Spam filter for SMS messages. 
- Our main public is telephone provider companies.
- Our main goal is achieve the best accuracy possible using the multinomial Naive Bayes algorithm. This is an supervised machine learning model. Naive Bayes algorithm is indicated when working with small datasets, it generalizes well with them, unlike more complex models like neural networks.
- To classify messages as spam or non-spam, the computer:
    1. Learns how humans classify messages;
    2. Uses that knowledge to estimate probabilities for new messages being spam or non-spam;
    3. Classifies a new message based on these values:
        - the probability for spam is greater — spam,
        - the probability for non-spam is greater — non-spam,
        - the two probability values are equal — we may need a human to classify the message.
- We're going to build the classificator, preparing the constants, the parameters and calculating the probabilites. In addition, we're going to use the Naive Bayes model of Scikit-learn library. In the end, we'll compare the accuracies to know which way performs better.

## Introducing the Data
- We'll use a dataset of 5,572 SMS that were already classified by humans. The dataset was put together by Tiago A. Almeida and José María Gómez Hidalgo, and it can be downloaded from the [UCI Machine Learning Repository.](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection)

In [1]:
# importing the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.style as style
import seaborn as sns

- The dataset doesn't have a header row, which means we need to use the header=None parameter, otherwise the first row will be wrongly used as the header row. We used the names=['Label', 'SMS'] parameter to name the columns as Label and SMS.

In [2]:
# importing the dataset
sms_set = pd.read_csv('SMSSpamCollection', sep='\t', header=None, names=['Label', 'SMS'])

In [3]:
# knowing the dataset
sms_set.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
# checking null values
sms_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Label   5572 non-null   object
 1   SMS     5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [5]:
# counting the labels
sms_set['Label'].value_counts()

ham     4825
spam     747
Name: Label, dtype: int64

In [6]:
# counting the percentage of the labels
sms_set['Label'].value_counts(normalize=True) * 100

ham     86.593683
spam    13.406317
Name: Label, dtype: float64

- In this first look at the dataset, we notice the absence of null values, and two types of label: "ham"(which means non-spam) with 86,6% of the rows and spam, with 13,4%.

### Training and Test set
- Next, before we start working on our filter, we need to design how we will test it. To do that, first we'll split our dataset into two categories:
    - A training set, that will be used to "train" the computer how to classify the messages;
    - A test set, that will be used to test the performance of our spam filter.
- Our training set will have 80% of the dataset (4,458 messages) and our test set will have 20% of the dataset (1,114 messages).

In [7]:
# randomize the dataset
data_randomized = sms_set.sample(frac=1, random_state=1)

# calculate index for split
training_test_index = round(len(data_randomized) * 0.8)

# training/Test split
training_set = data_randomized[:training_test_index].reset_index(drop=True) 
test_set = data_randomized[training_test_index:].reset_index(drop=True)
# drop parameter to avoid the old index being added as a column.

In [8]:
# checking the training set
training_set['Label'].value_counts(normalize=True) * 100

ham     86.54105
spam    13.45895
Name: Label, dtype: float64

In [9]:
# checking the test set
test_set['Label'].value_counts(normalize=True) * 100

ham     86.804309
spam    13.195691
Name: Label, dtype: float64

- As we want both splits of the dataset (the training and the test set) to be good representations of the whole dataset, we have to check if the percentages of the two types of label (ham and spam) are similar to the ones we found earlier. And we got arround 86% and 13% for both. 

## Data Cleaning
- On this section, we'll clean the dataset to prepare to the use of the Naive Bayes algorithm. First, we'll remove punctuation and turn lower case the wors in the messages. Then, transform each message in a list, create a vocabulary list with the unique words and create a dictionary with the word counts per sms. The final step is concatenate the DataFrame with the count of words and the training set.

In [10]:
# removing punctuation and turning lower case the words
training_set['SMS'] = training_set['SMS'].str.replace('\W', ' ').str.lower()
training_set.head()

C:\Users\Suporte\AppData\Local\Temp/ipykernel_1748/2879400168.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  training_set['SMS'] = training_set['SMS'].str.replace('\W', ' ').str.lower()


,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...


In [11]:
# transforming each message from the SMS column into a list
training_set['SMS'] = training_set['SMS'].str.split()
training_set.head()

,Label,SMS
0,ham,"[yep, by, the, pretty, sculpture]"
1,ham,"[yes, princess, are, you, going, to, make, me,..."
2,ham,"[welp, apparently, he, retired]"
3,ham,[havent]
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,..."


In [12]:
# crating a vocabulary list with the unique words of the column
vocabulary = []
for m in training_set['SMS']:
    for w in m:
        vocabulary.append(w)
vocabulary = list(set(vocabulary))

# number of unique words
len(vocabulary)

7783

In [13]:
# creating a dictionary with the word counts per sms
word_counts_per_sms = {unique_word: [0] * len(training_set['SMS']) for unique_word in vocabulary}
for index, sms in enumerate(training_set['SMS']):
    for word in sms:
        word_counts_per_sms[word][index]+=1
        
word_counts = pd.DataFrame(word_counts_per_sms)
word_counts.head()

,trouser,education,usmle,violated,miss,169,tag,tcr,realy,01223585334,...,worlds,diapers,hmph,edison,adventure,southern,2,name1,kickoff,worry
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,0,0,0


In [14]:
# concatenating the word_counts with the training_set
training_set_clean = pd.concat([training_set, word_counts], axis=1)
training_set_clean.head()

,Label,SMS,trouser,education,usmle,violated,miss,169,tag,tcr,...,worlds,diapers,hmph,edison,adventure,southern,2,name1,kickoff,worry
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,0,0,0


## Implementation
- Now, we're ready for the implementation of the Naive Bayes algorithm to classify the messages. It works upon the probability it gets from these two equations: ![](nbeq1.png)
- To calculate P(wi|Spam) and P(wi|Ham) inside the formulas we need other two equations: ![](nbeq2.png)
- where we need the constants:
    - Nwi|Spam</sub> — the number of times the word wi occurs in spam messages;
    - Nwi|Ham</sub> — the number of times the word wi occurs in ham messages;
    - NSpam — total number of words in spam messages;
    - NHam — total number of words in ham messages;
    - NVocabulary — total number of unique words in the vocabulary;
    - α — a smoothing parameter (Laplace smoother).

### 1. Calculating the constants
- We will start calculating constants for the Naive Bayes algorithm: probability of spam, probability of non-spam, number of words in spam messages, number of words in non-spam messages and total number of vocabulary. 

In [15]:
# Isolating spam and ham messages first
spam_messages = training_set_clean[training_set_clean['Label'] == 'spam']
ham_messages = training_set_clean[training_set_clean['Label'] == 'ham']

# P(Spam) and P(Ham)
p_spam = len(spam_messages) / len(training_set_clean)
p_ham = len(ham_messages) / len(training_set_clean)

# N_Spam
n_words_per_spam_message = spam_messages['SMS'].apply(len)
n_spam = n_words_per_spam_message.sum()

# N_Ham
n_words_per_ham_message = ham_messages['SMS'].apply(len)
n_ham = n_words_per_ham_message.sum()

# N_Vocabulary
n_vocabulary = len(vocabulary)

# Laplace smoothing
alpha = 1

### 2. Calculating the parameters
- Now that we have the constant terms calculated above, we can move on with calculating the parameters P(wi|Spam) and P(wi|Ham). Each parameter will thus be a conditional probability value associated with each word in the vocabulary.
- The Naive Bayes algorithm is very fast compared to other algorithms because we beforehand calculate the both probabilities for each individual word, which remains constant for every new message. When a new message comes in, most of the needed computations are already done, which enables the algorithm to almost instantly classify the new message.

In [16]:
# initiate parameters
parameters_spam = {unique_word:0 for unique_word in vocabulary}
parameters_ham = {unique_word:0 for unique_word in vocabulary}

# calculate parameters
for word in vocabulary:
    n_word_given_spam = spam_messages[word].sum()   # spam_messages already defined above
    p_word_given_spam = (n_word_given_spam + alpha) / (n_spam + alpha*n_vocabulary)
    parameters_spam[word] = p_word_given_spam
    
    n_word_given_ham = ham_messages[word].sum()   # ham_messages already defined above
    p_word_given_ham = (n_word_given_ham + alpha) / (n_ham + alpha*n_vocabulary)
    parameters_ham[word] = p_word_given_ham

### 3. Preparing the classificator
- Now, we're ready to create a function that classifies the new messages. After calculating the equations above, the classificator will compare the values of P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn), and return:
    - 'ham': if P(Ham|w1, w2, ..., wn) > P(Spam|w1, w2, ..., wn);
    - 'spam': if P(Ham|w1, w2, ..., wn) < P(Spam|w1, w2, ..., wn);
    - 'needs human classification': if P(Ham|w1, w2, ..., wn) = P(Spam|w1, w2, ..., wn).

In [17]:
# importing regular expressions
import re

# function of the classificator
def classify(message):
    """
    Classifies the given message based on the parameters previouly calculated and compares
    the two values and classifies the message as spam or ham, or requires 
    human classification. 
    
    Arg: message to be classified (str).
    
    Returns: classification of the message (str).
    """
    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]

        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]

    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_spam_given_message > p_ham_given_message:
        return 'spam'
    else:
        return 'needs human classification'

In [18]:
# testing the classification
print(test_set['SMS'][0])
print(classify(test_set['SMS'][0]))

print(test_set['SMS'][30])
print(classify(test_set['SMS'][30]))

Later i guess. I needa do mcat study too.
ham
Get your garden ready for summer with a FREE selection of summer bulbs and seeds worth £33:50 only with The Scotsman this Saturday. To stop go2 notxt.co.uk
spam


## Measuring the accuracy
- In this section, we'll use the classificator in the test set, create a new column called 'Predicted' and calculate the accuracy of our spam filter.

In [19]:
# creating the new column 'Predicted' with the results of the classification
test_set['Predicted'] = test_set['SMS'].apply(classify)
test_set.head()

,Label,SMS,Predicted
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


In [20]:
# Calculating the accuracy of the spam filter
correct = 0
total = len(test_set)        # number of sms in the test set
for row in test_set.iterrows():
    if row[1]['Predicted']==row[1]['Label']:
        correct+=1
accuracy = correct/total*100

print(correct)
print(total)
print(round(accuracy, 2))

1100
1114
98.74


- In 1114 classified messages, 1100 were correct. It is a very good number, 98.74%. In the next section, we'll look at the messages that were incorrectly classified (14) to understand the errors.

## Analysing the errors
- Let's read and analyse the 14 messages that got a wrong classification. We'll divide them into three categories: 
    - False spam: the ones classificated as 'spam', but were 'ham';
    - False ham: the ones classificated as 'ham', but were 'spam';
    - Needs human classification: the ones that the classificator couldn't identify.

In [21]:
# classificating the wrong results
false_spam = test_set[(test_set['Predicted']=='spam')&(test_set['Label']=='ham')]
false_ham = test_set[(test_set['Predicted']=='ham')&(test_set['Label']=='spam')]
needs_human_classification = test_set[test_set['Predicted']=='needs human classification']

In [22]:
# checking 'false spam'
print(false_spam)

    Label                                                SMS Predicted
152   ham                  Unlimited texts. Limited minutes.      spam
159   ham                                       26th OF JULY      spam
284   ham                             Nokia phone is lovly..      spam
302   ham                   No calls..messages..missed calls      spam
319   ham  We have sent JD for Customer Service cum Accou...      spam


In [23]:
# checking 'false ham'
print(false_ham)

    Label                                                SMS Predicted
114  spam  Not heard from U4 a while. Call me now am here...       ham
135  spam  More people are dogging in your area now. Call...       ham
504  spam  Oh my god! I've found your number again! I'm s...       ham
546  spam  Hi babe its Chloe, how r u? I was smashed on s...       ham
741  spam  0A$NETWORKS allow companies to bill for SMS, s...       ham
876  spam           RCT' THNQ Adrian for U text. Rgds Vatian       ham
885  spam                                      2/2 146tf150p       ham
953  spam  Hello. We need some posh birds and chaps to us...       ham


In [24]:
# checking false 'needs human classification'
print(needs_human_classification)

    Label                                                SMS  \
293   ham  A Boy loved a gal. He propsd bt she didnt mind...   

                      Predicted  
293  needs human classification  


- Analysing the wrong cases, we got five false spam (when the classification is spam, but actually is ham), eight false ham (the opposite case, when the predicted label gets ham, but it is spam) and one case of unclear classification (needs human classification). 
- The lenght of the messages can affect the classification. Ususally short messages are labeled as spam. And the presence of suspicious ad-style words, like "call", "contact", "free" or "phone", can affect the precision of the classificator. A third aspect that can affect the accuracy is the absence of the words in the new message in the vocabulary of the classificator. We think that was the reason of the case of unclear classification. This message is long and full of slangs and abbreviations that probably aren't in the vocabulary.

## Using Scikit-learn library
- In this section, we'll keep the same model, but using Scikit-learn library. This is one of the most popular machine learning libraries. Scikit-learn contains five types of the Naive Bayes model: the Gaussian, the Bernoulli, the Multinomial, the Complement and the Categorical.
- We'll use the Multinomial, the most indicated one for document or text classification.

In [25]:
# make a copy of the set
skl_set = sms_set.copy()

- To run the model in the library, we need to convert the label column into numeric:

In [26]:
# converting the labels to numeric
skl_set['Label'] = np.where(skl_set['Label']=='spam',1, 0)
skl_set.head()

,Label,SMS
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [27]:
# importing the methods, functions and classes
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

- Now, we're going to split the data into training and test set. By default, sklearn splits in the ratio of 70:30, but we're changing into 80:20 to make it more similar to the classificator we build above.

In [28]:
# splitting the data
X_train, X_test, Y_train, Y_test = train_test_split(skl_set['SMS'], 
                                                    skl_set['Label'], 
                                                    random_state=0,
                                                    test_size=0.2)

- Our next step is transform the messages to the vectorized form, using the CountVectorizer class. We need to do this beacuse the machine learning algorithms works doing computation, which is possible with numerical values.
- We also will change the regular expression that filters the message, again to look similar to our previous classificator.

In [29]:
# extracting features
vectorizer = CountVectorizer(token_pattern='\W').fit(X_train)
X_train_vectorized = vectorizer.transform(X_train)
X_train_vectorized.toarray().shape

(4457, 45)

- Now, we train the instantiate the model (MultinomialNB) and fit the model to the training data. In addition, we'll set the alpha to 1, like the previous classificator.

In [32]:
# instantiate the model and fit the trainng data
model = MultinomialNB(alpha=1)
model.fit(X_train_vectorized, Y_train)

MultinomialNB(alpha=1)

- The next step is to use the model to make predictions and evaluate the accuracy.

In [33]:
# predict and measure the accuracy
predictions = model.predict(vectorizer.transform(X_test))
print("Accuracy:", 100 * sum(predictions == Y_test) / len(predictions), '%')

Accuracy: 94.17040358744394 %


- Doing this way, we got 94,17% of accuracy, less than our previous implementation with 98,74%. Let's change the parameters we choose to see if we can improve our accuracy.

### Changing parameters
- We'll use the same model before, but now with some changes:
    - The default train/test ratio: 70:30;
    - The default token_pattern of the CountVectorized class: r"(?u)/b/W/W+b";
    - Unigram and bigram in the CountVectorized class: ngram_range=(1, 2);
    - Alpha = 0.1.

In [37]:
# splitting the data
X_train, X_test, Y_train, Y_test = train_test_split(skl_set['SMS'], 
                                                    skl_set['Label'], 
                                                    random_state=0)

# extracting features
vectorizer = CountVectorizer(ngram_range=(1, 2)).fit(X_train)
X_train_vectorized = vectorizer.transform(X_train)
X_train_vectorized.toarray().shape

# instantiate the model and fit the trainng data
model = MultinomialNB(alpha=0.1)
model.fit(X_train_vectorized, Y_train)

# predict and measure the accuracy
predictions = model.predict(vectorizer.transform(X_test))
print("Accuracy:", 100 * sum(predictions == Y_test) / len(predictions), '%')

Accuracy: 98.77961234745155 %


- We got an accuracy of 98.77%, the best among the three classifications. The first one with 98,74% and the second with 94.17%.

## Conclusion

- In this project, we created a spam filter based on the Naive Bayes algorithm (multinomial type). After test three ways of classification, we got one using the scikit-learn library that reaches 98.77% of accuracy, a high number.
- The filter takes a message and classifies as spam or ham. 
- Some of the advantages of this filter are: it can be trained on a per-user basis and it can perform particularly well in avoiding false positives.
- The disavantages are: some filters could be susceptible to Bayesian poisoning (when a spammer send out emails with large amounts of legitimate text), spammers can transform the words used normally in large quantities and another technique used is to replace text with pictures, either directly included or linked.
- More information about Naive Bayes spam filter can be found [here](https://en.wikipedia.org/wiki/Naive_Bayes_spam_filtering#Discussion).